<a href="https://colab.research.google.com/github/immin0241/school_projects/blob/master/3_1_ai/wip_ai_jobs_wage_expectation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

df = pd.read_csv('/content/ai_job_dataset.csv')

korea_df = df[df['company_location'] == 'South Korea'].copy()

features = ['required_skills', 'experience_level', 'years_experience']
target = 'salary_usd'

X = korea_df[features]
y = korea_df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"훈련 데이터 수: {len(X_train)}개, 테스트 데이터 수: {len(X_test)}개")

text_transformer = CountVectorizer(tokenizer=lambda x: [i.strip() for i in x.split(',')], max_features=300, binary=True)

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

numeric_transformer = MinMaxScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('text', text_transformer, 'required_skills'),
        ('cat', categorical_transformer, ['experience_level']),
        ('num', numeric_transformer, ['years_experience'])    ],
    remainder='passthrough' )


model = Sequential([
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])


pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', model)])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = pipeline.fit(X_train, y_train,
                       regressor__epochs=200,
                       regressor__batch_size=16,
                       regressor__validation_split=0.2,
                       regressor__callbacks=[early_stopping],
                       regressor__verbose=1)

# Preprocess X_test before evaluating the model
X_test_processed = pipeline.named_steps['preprocessor'].transform(X_test)

loss, mae = model.evaluate(X_test_processed, y_test, verbose=0)
print(f"\n테스트 데이터에 대한 최종 Mean Absolute Error (MAE): ${mae:,.2f}")
print(f"-> 모델의 연봉 예측치가 실제값과 평균적으로 ${mae:,.2f} 정도 차이남을 의미합니다.")


predictions = pipeline.predict(X_test)

results_df = pd.DataFrame({
    'Actual Salary': y_test.values.flatten(),
    'Predicted Salary': predictions.flatten()
})
results_df['Difference'] = results_df['Actual Salary'] - results_df['Predicted Salary']

print("\n--- 실제 연봉과 예측 연봉 비교 (상위 5개) ---")
print(results_df.head())

훈련 데이터 수: 577개, 테스트 데이터 수: 145개
Epoch 1/200


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 89105.2969 - mae: 89105.2969 - val_loss: 88673.5781 - val_mae: 88673.5781
Epoch 2/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 86264.7344 - mae: 86264.7344 - val_loss: 88639.9141 - val_mae: 88639.9141
Epoch 3/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 83414.6406 - mae: 83414.6406 - val_loss: 88441.3906 - val_mae: 88441.3906
Epoch 4/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 89592.2891 - mae: 89592.2891 - val_loss: 87711.5781 - val_mae: 87711.5781
Epoch 5/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 85169.0859 - mae: 85169.0859 - val_loss: 85804.7734 - val_mae: 85804.7734
Epoch 6/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 82521.7656 - mae: 82521.7656 - val_loss: 81732.0547 - val_mae: 81732.0547
Epoch 7/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 80607.3281 - mae: 80607.3281 - val_loss: 74246.9062 - val_mae: 74246.9062
Epoch 8/200
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 69340.7500 - mae: 